# 251 HW9 - ImageNet Single Instance/GPU

#### Import Modules

In [1]:
!pip install wandb -qqq

In [2]:
import os
import random
import shutil
import time
import warnings
import subprocess

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from torch.cuda.amp import autocast, GradScaler

import wandb

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Weights & Biases Setup

In [3]:
wandb.login()
wandb.init(project="251-lab9")

wandb: Currently logged in as: hsiungc. Use `wandb login --relogin` to force relogin


#### Set Random Seed

In [4]:
SEED=1

In [5]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

#### Hyperparameters & Variables

In [6]:
START_EPOCH = 0

In [7]:
ARCH = 'resnet18'
EPOCHS = 1
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=784
VAL_BATCH=784
WORKERS=2
TRAINDIR="/data/ebs/train"
VALDIR="/data/ebs/val"

In [8]:
best_acc1 = 0

#### Set CUDA Device

In [9]:
torch.cuda.device_count()

1

In [10]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [11]:
GPU = 'cuda:0'

In [12]:
cudnn.benchmark = True

## Functions

#### NVIDIA SMI

In [13]:
def get_gpu_utilization():
    result = subprocess.run(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv'], stdout=subprocess.PIPE)
    output = result.stdout.decode('utf-8')
    return output

#### Training & Validation

In [14]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    scaler = GradScaler()
        
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        images = images.cuda(GPU, non_blocking=True)
        target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast(dtype=torch.float16):
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        wandb.log({'accuracy1': acc1, 'accuracy5': acc5, 'loss': loss, 'learningrate': LR}, step=i)
            
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [15]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        
        for i, (images, target) in enumerate(val_loader):
            images = images.cuda(GPU, non_blocking=True)
            target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            wandb.log({'val_accuracy1': acc1, 'val_accuracy5': acc5, 'val_loss': loss, 'val_learningrate': LR}, step=1)            
            
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

    return top1.avg

#### Checkpoint & Progress

In [16]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [17]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [18]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

#### Learning Rate & Accuracy

In [19]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [20]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

#### Normalization & Transformations

In [21]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [22]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    #transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [23]:
transform_val = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

#### Dataset & DataLoader

In [24]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [25]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [26]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [27]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

#### Model Setup

In [28]:
model = models.__dict__[ARCH](pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
model = model.cuda(GPU)

#### Loss & Optimizer

In [30]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [31]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [32]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

#### Training & Validation Loop

In [33]:
for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    utilization = get_gpu_utilization()
    print(utilization)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

wandb.finish()

Epoch: [0][   0/1635]	Time  8.537 ( 8.537)	Data  4.359 ( 4.359)	Loss 1.0076e+00 (1.0076e+00)	Acc@1  72.70 ( 72.70)	Acc@5  92.47 ( 92.47)
Epoch: [0][  50/1635]	Time  2.623 ( 1.866)	Data  2.187 ( 1.356)	Loss 9.8724e-01 (9.7196e-01)	Acc@1  75.51 ( 75.13)	Acc@5  91.45 ( 92.43)
Epoch: [0][ 100/1635]	Time  3.894 ( 2.127)	Data  3.458 ( 1.653)	Loss 9.5116e-01 (9.4817e-01)	Acc@1  76.15 ( 75.76)	Acc@5  92.22 ( 92.68)
Epoch: [0][ 150/1635]	Time  3.423 ( 2.306)	Data  2.987 ( 1.844)	Loss 9.1579e-01 (9.3560e-01)	Acc@1  75.38 ( 75.95)	Acc@5  92.35 ( 92.83)
Epoch: [0][ 200/1635]	Time  4.172 ( 2.393)	Data  3.736 ( 1.938)	Loss 9.5504e-01 (9.2757e-01)	Acc@1  76.28 ( 76.13)	Acc@5  93.11 ( 92.95)
Epoch: [0][ 250/1635]	Time  4.523 ( 2.430)	Data  4.087 ( 1.979)	Loss 8.2480e-01 (9.2096e-01)	Acc@1  79.97 ( 76.30)	Acc@5  94.01 ( 93.04)
Epoch: [0][ 300/1635]	Time  4.606 ( 2.451)	Data  4.170 ( 2.002)	Loss 8.6754e-01 (9.1565e-01)	Acc@1  78.70 ( 76.37)	Acc@5  92.98 ( 93.10)
Epoch: [0][ 350/1635]	Time  3.056 ( 2.462

accuracy1,▃▃▄▃▁▆▂▂▃▂▁▃▅▁▄▄▂▇▅▄▇▅▃▅▆▅▄▃▃▄▄▄▅█▅▃▆▅▃▅
accuracy5,▃▆▂▁▁▅▅▂▅▄▄▄▃▃▇▅▄▄▄▁▅▄▃▆█▃▃▅▅▂▅▇▇█▆▄▄▆▅▃
learningrate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▅▅▆▅█▄▅▆▄▅▆▄▄▆▄▄▅▃▄▅▃▄▅▃▁▃▅▄▃▄▄▃▂▁▂▅▄▃▄▄
accuracy1,81.98198
accuracy5,92.79279
learningrate,0.001
loss,0.8003
